In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# ✅ AGI1_05d_text_pretraining.ipynb: Text Pretraining with Memory (RAG-lite)

import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datasets import Dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Using device: {device}")

# ✅ Step 1: Load dataset
csv_path = '/kaggle/input/alpaca-dataset-v1/alpaca_data_cleaned.csv'
df = pd.read_csv(csv_path)
print(f"✅ Loaded dataset: {len(df)} examples")

# ✅ Step 2: Create memory via embeddings
def format_text(row):
    return f"Instruction: {row['instruction']}\nInput: {row.get('input', '')}\nOutput: {row['output']}"

df['formatted'] = df.apply(format_text, axis=1)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = []
for text in df['formatted']:
    chunks += text_splitter.split_text(text)

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_texts(chunks, embedding_model)
print("✅ FAISS memory built with embeddings")

db.save_local("/kaggle/working/agi_memory_faiss")

# ✅ Step 3: Load LLM
model_name = 'distilgpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# ✅ Step 4: Prepare dataset for pretraining
df['text'] = df['formatted']

dataset = Dataset.from_pandas(df[['text']])
dataset = dataset.train_test_split(test_size=0.1)

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# ✅ Step 5: Training arguments
training_args = TrainingArguments(
    output_dir='/kaggle/working/agi1_05d_output',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=5
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

# ✅ Step 6: Save model and memory
trainer.save_model('/kaggle/working/agi1_05d_output')
tokenizer.save_pretrained('/kaggle/working/agi1_05d_output')
print("✅ Fine-tuned model saved to /kaggle/working/agi1_05d_output")

2025-07-14 10:15:57.645027: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752488157.860921      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752488157.925921      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


ModuleNotFoundError: Module langchain_community.vectorstores not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`